In [1]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import dict_map as dm

filepath = './models/l300e50wWeights'
# Load the model
model = load_model(filepath, compile = True)

In [2]:
df = pd.read_csv('data/trainingdata300.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)
# df = df.head(1000)
token2idx, idx2token = dm.get_dict_map(df, 'token')
tag2idx, idx2tag = dm.get_dict_map(df, 'tag')

df['Word_idx'] = df['Wort'].map(token2idx)
df['Tag_idx'] = df['Attribut'].map(tag2idx)

df_group = df.groupby(by = ['satzId'], as_index=False)['Wort', 'Attribut', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
#df_group
import pad_train_test as ptt
train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = ptt.get_pad_train_test_val(df_group, df, tag2idx)

<ipython-input-2-ffa0064ed6ee>:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_group = df.groupby(by = ['satzId'], as_index=False)['Wort', 'Attribut', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))


train_tokens length: 26331 
train_tags length: 26331 
test_tokens length: 3902 
test_tags: 3902 
val_tokens: 8778 
val_tags: 8778


In [ ]:
df_zu_testzwecken = pd.DataFrame(data=[0,1,2,3,4,5,6,7], columns=['IDX'])
df_zu_testzwecken['Tag'] = df_zu_testzwecken['IDX'].map(idx2tag)
print(df_zu_testzwecken)

In [3]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_tokens, np.array(test_tags), batch_size=512)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(test_tokens[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
8/8 [==============================] - 27s 3s/step - loss: 7.0791 - accuracy: 0.0028
test loss, test acc: [7.079083442687988, 0.0028267554007470608]
Generate predictions for 3 samples
predictions shape: (3, 300, 8)


In [ ]:
predictions

In [5]:
classes = np.argmax(predictions, axis = 1)

In [6]:
# print(predictions)
print(classes)

[[  0   0 103   0   0   0   0   0]
 [  0   0 192   0   0   0   0   0]
 [  0   0 186   0   0   0   0   0]]


In [13]:
df.groupby('Attribut').count()

,satzId,Wort,Word_idx,Tag_idx
Attribut,,,,
B-Brand,67464,67464,67464,67464
B-Modelnumber,33296,33115,33296,33296
E-Brand,4521,4521,4521,4521
E-Modelnumber,7143,7143,7143,7143
I-Brand,1068,1068,1068,1068
I-Modelnumber,7123,7123,7123,7123
O,4807866,4807703,4807866,4807866


In [7]:
df.isna().sum()

satzId        0
Wort        344
Attribut      3
Word_idx      0
Tag_idx       0
dtype: int64